In [8]:
import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial
import PIL
import PIL.Image
import random as python_random
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(42) # note that you must use the same seed to ensure consistentcy in your training/validation/testing
tf.random.set_seed(42)



#### Questions:
# How does tf (under this method) store labels? Can I access them?
#    Look into how the resizing line is done
# What happens if I don't pass the y value into imageDataGenerator().flow()?
# is this enough work? (oversampling, data augmentation, adusting the prediction wieghts)
# multiple expert -  3 different model (majority voting at the end to make predictions) - cost trade off for computing
# - talk about costs (training time, different work, etc)
# - accuracy vs tradeoffs (latency of making one prediction)

## Class Analysis

In [9]:
!ls

README.md	  jordan_cnn-manual-split.ipynb  resnet-jordan.ipynb
checkpoints	  jordan_cnn.ipynb		 resnet.ipynb
dl_load.ipynb	  matrix1.png			 resnet_v2.ipynb
evaluation.ipynb  matrix2.png			 saved_models


In [10]:
data_path = '../dl_data/'
class_names = os.listdir(data_path)
class_dist = {}
for c in class_names:
    class_dist[c] = len(os.listdir(data_path + c))
    print(c)
    print(class_dist[c])

Covid_img
3611
Viral_img
1345
Normal_img
10193


In [11]:
import math

def pull_val(dir_path, val_path, portion):
    # get a list of files to hold out for validation
    files = os.listdir(dir_path)
    length = len(files)
    num_files = math.floor(length * portion)
    val_files = np.random.choice(files, size=num_files, replace=False)

    # move files
    for f in val_files:
        os.rename(dir_path + '/' + f, val_path + '/' + f)

In [12]:
# if you need to, use this to make a new directory

# os.mkdir('../HOLD_data')
# os.mkdir('../test_data')
# for c in class_names:
#     os.mkdir('../HOLD_data/' + c)
#     os.mkdir('../test_data/' + c)

In [13]:
# save out holdout and testing data

portion = .10 # portion of data set aside for HOLDOUT
    
for c in class_names:
    dir_path = '../dl_data/' + c
    val_path = '../HOLD_data/' + c
    pull_val(dir_path, val_path, portion)
    
##### Commented out because we will do validation split with the image gen function    
# portion = .20 # portion of data set aside for TESTING
    
# for c in class_names:
#     dir_path = '../dl_data/' + c
#     val_path = '../test_data/' + c
#     pull_val(dir_path, val_path, portion)

In [14]:
# to reset holdout

# for c in class_names:
#     val_files = os.listdir('../HOLD_data/' + c)
#     for i in val_files:
#         os.rename('../HOLD_data/' + c + '/' + i, '../dl_data/' + c + '/' + i)
        
# for c in class_names:
#     val_files = os.listdir('../test_data/' + c)
#     for i in val_files:
#         os.rename('../test_data/' + c + '/' + i, '../dl_data/' + c + '/' + i)

In [15]:
print('---Train set-------------------')
for c in class_names:
    print(c)
    print(len(os.listdir('../dl_data/'+c)))
    
print('---Teset set-------------------')
for c in class_names:
    print(c)
    print(len(os.listdir('../test_data/'+c)))
    
print('---Holdout set-------------------')
for c in class_names:
    print(c)
    print(len(os.listdir('../HOLD_data/'+c)))
    
print('---Total-------------------')
for c in class_names:
    print(c)
    print(len(os.listdir('../dl_data/'+c)) + len(os.listdir('../HOLD_data/'+c)) + len(os.listdir('../test_data/'+c)))

---Train set-------------------
Covid_img
3250
Viral_img
1211
Normal_img
9174
---Teset set-------------------
Covid_img
0
Viral_img
0
Normal_img
0
---Holdout set-------------------
Covid_img
361
Viral_img
134
Normal_img
1019
---Total-------------------
Covid_img
3611
Viral_img
1345
Normal_img
10193


## Load Data

In [16]:
# if the data above isn't run, run this

data_path = "../dl_data/"
class_names = os.listdir(data_path)
class_dist = {}
for c in class_names:
    class_dist[c] = len(os.listdir(data_path + c))

In [17]:
class_names

['Covid_img', 'Viral_img', 'Normal_img']

In [23]:
from sklearn.datasets import load_files 
from keras.utils import np_utils

from keras.preprocessing import image

#### calculate class weights

# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
total = sum(class_dist.values())
weight_for_0 = (1 / class_dist[class_names[0]]) * (total / 2.0)
weight_for_1 = (1 / class_dist[class_names[1]]) * (total / 2.0)
weight_for_2 = (1 / class_dist[class_names[2]]) * (total / 2.0)

class_weights = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

# directories
data_dir = "../dl_data"
# test_dir = "../test_data"
HOLD_dir = "../HOLD_data"


#### save out augmented data for visualization

# ## first delete any existing files
# aug_dir = '../augmented_data'
# aug_files = os.listdir(aug_dir)
# for f in aug_files:
#     os.remove(aug_dir + '/' + f)

    
batch_size = 32;
# IMPORTANT: Depends on what pre-trained model you choose, you will need to change these dimensions accordingly
img_height = 224; 
img_width = 224;
    

# data augmentation (for training only)
train_data_gen = ImageDataGenerator(rescale=1./255,
                                    zoom_range= 0.3, 
                                    horizontal_flip= True, 
                                    shear_range= 0.2,
                                    rotation_range = 30,
                                    validation_split=0.2
                                    
                                    
#                                     featurewise_center=False,
#                                     samplewise_center=False,
#                                     featurewise_std_normalization=False,
#                                     samplewise_std_normalization=False,
#                                     zca_whitening=False,
#                                     zca_epsilon=1e-06,
#                                     rotation_range=0,
#                                     width_shift_range=0.0,
#                                     height_shift_range=0.0,
#                                     brightness_range=None,
#                                     shear_range=0.0,
#                                     zoom_range=0.0,
#                                     channel_shift_range=0.0,
#                                     fill_mode='nearest',
#                                     cval=0.0,
#                                     horizontal_flip=False,
#                                     vertical_flip=False,
#                                     rescale=None,
#                                     preprocessing_function=None,
#                                     data_format=None,
#                                     validation_split=0.2,
#                                     dtype=None
                                    )


train_ds = train_data_gen.flow_from_directory(
    directory = data_dir,
    target_size=(img_height, img_width),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42,
#     save_to_dir=aug_dir,
#     save_prefix='aug',
#     save_format='png',
    follow_links=False,
    subset='training',
    interpolation='nearest'
)

validation_ds = train_data_gen.flow_from_directory(
    directory=data_dir,  # same directory because we are splitting the data here
    follow_links=False,
    subset='validation',
    interpolation='nearest',
    target_size=(img_height, img_width), 
    class_mode='categorical',
    shuffle=True,
    seed=42,
    batch_size=batch_size
)


test_data_gen = ImageDataGenerator(rescale=1./255)


# holdout data
HOLD_ds = test_data_gen.flow_from_directory(directory=HOLD_dir, 
                                         target_size=(img_height, img_width), 
                                         class_mode='categorical',
                                         shuffle=True,
                                         seed=42,
                                         batch_size=batch_size)

Found 10912 images belonging to 3 classes.
Found 2726 images belonging to 3 classes.
Found 1514 images belonging to 3 classes.


In [24]:
## this is just a bug fix, hopefully I won't need to use it again.

# fi = os.listdir(aug_dir + '/' + os.listdir(aug_dir)[0])
# for f in fi:
#     os.remove(aug_dir + '/' + os.listdir(aug_dir)[0] + '/' + f)

# os.rmdir(aug_dir + '/' + os.listdir(aug_dir)[0])

In [25]:
# Visualize some of the train samples of one batch
# Make sure you create the class names that match the order of their appearances in the "files" variable
class_names = ['Covid',  'Normal',  'Viral']

# Rows and columns are set to fit one training batch (32)
n_rows = 8
n_cols = 4
plt.figure(figsize=(n_cols * 3, n_rows * 3))
for images, labels in train_ds.take(1):
    for i in range (n_rows*n_cols):
        plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis('off')
        plt.title(class_names[labels[i]], fontsize=12)
plt.subplots_adjust(wspace=.2, hspace=.2)

AttributeError: 'DirectoryIterator' object has no attribute 'take'

<Figure size 864x1728 with 0 Axes>

In [26]:
# set checkpoint to resume training if it stops unexpectedly
checkpoint_path = "../checkpoints/training_2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [27]:
ds_size_1 = (224, 224)
# train_ds_1 = train_ds.map(lambda image, label: (tf.image.resize(image, ds_size_1), label))
# validation_ds_1 = validation_ds.map(lambda image, label: (tf.image.resize(image, ds_size_1), label))

train_ds_1 = train_ds
validation_ds_1 = validation_ds


base_model_2 = keras.applications.ResNet50(weights='imagenet', include_top=False)
n_classes = 3

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# Rebuild top
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(base_model_2.output)
x = tf.keras.layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
# x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(3, activation="softmax", name="pred")(x) # match number of classes

model_2 = keras.models.Model(inputs=base_model_2.input,
                           outputs=outputs)

In [29]:
# train up the top layer first

for layer in base_model_2.layers:
    layer.trainable = False

optimizer = keras.optimizers.Adam(learning_rate=0.01, decay=0.01)
model_2.compile(loss="categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model_2.fit(train_ds_1,
                    validation_data=validation_ds_1,
#                     class_weight=class_weights,
                    epochs=20, callbacks=[callback,cp_callback])

Epoch 1/20
341/341 [==============================] - ETA: 0s - loss: 0.6408 - accuracy: 0.7106
Epoch 1: saving model to ../checkpoints/training_2/cp.ckpt
341/341 [==============================] - 180s 519ms/step - loss: 0.6408 - accuracy: 0.7106 - val_loss: 0.8320 - val_accuracy: 0.6728
Epoch 2/20
341/341 [==============================] - ETA: 0s - loss: 0.5639 - accuracy: 0.7407
Epoch 2: saving model to ../checkpoints/training_2/cp.ckpt
341/341 [==============================] - 176s 517ms/step - loss: 0.5639 - accuracy: 0.7407 - val_loss: 0.5668 - val_accuracy: 0.7432
Epoch 3/20
341/341 [==============================] - ETA: 0s - loss: 0.5555 - accuracy: 0.7489
Epoch 3: saving model to ../checkpoints/training_2/cp.ckpt
341/341 [==============================] - 174s 510ms/step - loss: 0.5555 - accuracy: 0.7489 - val_loss: 0.5584 - val_accuracy: 0.7612
Epoch 4/20
341/341 [==============================] - ETA: 0s - loss: 0.5477 - accuracy: 0.7565
Epoch 4: saving model to ../checkp

In [30]:
# opt = tf.keras.optimizers.Adam(0.1)
# net = Net()
# dataset = toy_dataset()
# iterator = iter(dataset)
# ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=net, iterator=iterator)
# manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=3)

# train_and_checkpoint(net, manager)

In [ ]:
# train all the layers together for a bit with a much lower learning rate

for layer in base_model_2.layers[-20:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

optimizer = keras.optimizers.Adam(learning_rate=0.0004, decay=0.001)
model_2.compile(loss="categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model_2.fit(train_ds_1,
                    validation_data=validation_ds_1,
#                     class_weight=class_weights,
                    epochs=50, callbacks=[callback,cp_callback])

Epoch 1/50
341/341 [==============================] - ETA: 0s - loss: 0.7362 - accuracy: 0.6940
Epoch 1: saving model to ../checkpoints/training_2/cp.ckpt
341/341 [==============================] - 179s 515ms/step - loss: 0.7362 - accuracy: 0.6940 - val_loss: 13.9143 - val_accuracy: 0.2384
Epoch 2/50
341/341 [==============================] - ETA: 0s - loss: 0.6195 - accuracy: 0.7337
Epoch 2: saving model to ../checkpoints/training_2/cp.ckpt
341/341 [==============================] - 174s 511ms/step - loss: 0.6195 - accuracy: 0.7337 - val_loss: 94.1256 - val_accuracy: 0.0888
Epoch 3/50
341/341 [==============================] - ETA: 0s - loss: 0.5934 - accuracy: 0.7501
Epoch 3: saving model to ../checkpoints/training_2/cp.ckpt
341/341 [==============================] - 175s 513ms/step - loss: 0.5934 - accuracy: 0.7501 - val_loss: 25.7096 - val_accuracy: 0.2384
Epoch 4/50
341/341 [==============================] - ETA: 0s - loss: 0.5596 - accuracy: 0.7636
Epoch 4: saving model to ../che

In [ ]:
# save the model
model_2.save('saved_models/model_TEST') # change this path to save a new version

In [ ]:
# # if you need to use the checkpoint, use this code
# # source: https://www.tensorflow.org/tutorials/keras/save_and_load#checkpoint_callback_options

# latest = tf.train.latest_checkpoint(checkpoint_dir)
# latest

# # Create a new model instance
# model_2 = create_model()

# # Load the previously saved weights
# model_2.load_weights(latest)

# # Re-evaluate the model
# loss, acc = model_2.evaluate(validation_ds_1 verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
! ls 

## Model 1
Note this is model was trained on train/test data before validation (holdout data was taken out)

In [7]:
# predictions_1 = model_1.predict(validation_ds_1, batch_size=batch_size)
# len(predictions_1)

# or get predeictions from saved model

saved_model_1 = tf.keras.models.load_model('./saved_models/model_1')
predictions_1 = saved_model_1.predict(validation_ds)

In [11]:
validation_ds

In [9]:
y_true = []
for img, label in validation_ds:
    y_true.append(label.numpy())
y_true

# create and plot confusion matrix
confusion_mtx = tf.math.confusion_matrix(
    np.concatenate(y_true),
    np.argmax(predictions_1, axis=1),
    num_classes=3,
    weights=None) # change to get to 100% covid accuracy
# confusion_mtx = confusion_mtx/confusion_mtx.numpy().sum(axis=1)[:, tf.newaxis]
plt.figure(figsize=(10, 8))
sns.set(font_scale = 2)
sns.heatmap(confusion_mtx,
            xticklabels=class_names,
            yticklabels=class_names,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.savefig('matrix1.png')
plt.show()

KeyboardInterrupt: 

### On holdout data

In [6]:
# HOLD_ds_1 = HOLD_ds.map(lambda image, label: (tf.image.resize(image, ds_size_1), label))
HOLD_predictions_1 = saved_model_1.predict(HOLD_ds)

y_true = []
for img, label in HOLD_ds:
    y_true.append(label)
y_true

# create and plot confusion matrix
confusion_mtx = tf.math.confusion_matrix(
    np.concatenate(y_true),
    np.argmax(HOLD_predictions_1, axis=1),
    num_classes=3,
    weights=None) # change to get to 100% covid accuracy
plt.figure(figsize=(10, 8))
sns.set(font_scale = 2)
sns.heatmap(confusion_mtx,
            xticklabels=class_names,
            yticklabels=class_names,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.savefig('matrix1.png')
plt.show()

KeyboardInterrupt: 

## Model 0
This model was created in the resnet_v2 file.


In [ ]:
# or get predeictions from saved model

saved_model_0 = tf.keras.models.load_model('./saved_models/model_0')
predictions_0 = saved_model_0.predict(validation_ds_1)

In [ ]:
y_true = []
for img, label in validation_ds_1:
    y_true.append(label.numpy())
y_true

# create and plot confusion matrix
confusion_mtx = tf.math.confusion_matrix(
    np.concatenate(y_true),
    np.argmax(predictions_0, axis=1),
    num_classes=3,
    weights=None) # change to get to 100% covid accuracy
plt.figure(figsize=(10, 8))
sns.set(font_scale = 2)
sns.heatmap(confusion_mtx,
            xticklabels=class_names,
            yticklabels=class_names,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.savefig('matrix1.png')
plt.show()

In [ ]:
predictions_0

## Model 2

In [ ]:
# predictions_1 = model_1.predict(validation_ds_1, batch_size=batch_size)
# len(predictions_1)

# or get predeictions from saved model

saved_model_2 = tf.keras.models.load_model('./saved_models/model_2')
predictions_2 = saved_model_2.predict(validation_ds_1)

In [ ]:
y_true = []
for img, label in validation_ds_1:
    y_true.append(label.numpy())
y_true

# create and plot confusion matrix
confusion_mtx = tf.math.confusion_matrix(
    np.concatenate(y_true),
    np.argmax(predictions_2, axis=1),
    num_classes=3,
    weights=None) # change to get to 100% covid accuracy
plt.figure(figsize=(10, 8))
sns.set(font_scale = 2)
sns.heatmap(confusion_mtx,
            xticklabels=class_names,
            yticklabels=class_names,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.savefig('matrix1.png')
plt.show()

### On Holdout Data

In [ ]:
y_true = []
for img, label in validation_ds_1:
    y_true.append(label.numpy())
y_true

# create and plot confusion matrix
confusion_mtx = tf.math.confusion_matrix(
    np.concatenate(y_true),
    np.argmax(predictions_2, axis=1),
    num_classes=3,
    weights=None) # change to get to 100% covid accuracy
plt.figure(figsize=(10, 8))
sns.set(font_scale = 2)
sns.heatmap(confusion_mtx,
            xticklabels=class_names,
            yticklabels=class_names,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.savefig('matrix1.png')
plt.show()

To Do:

Oversampling/Data Augmentation:

1. start a new file with clear labels, resampling, augmented data
2. Train the model the same way
3. Save model and create confusion matrix in this file (or seperate file)

Prediction weights
1. When predicting classes, change wieghts until we get 100% for covid cases
2. Change to proportional CM instead of just numeric?

Recall and F-score as metric?